In [1]:
import tensorflow as tf
import pandas as pd
df = pd.read_csv("initial_dataset_main_sets.csv")
#df.head(100)

In [2]:
df2 = df.query("season > 11")
df2.reset_index(inplace=True,drop=True)
#df2.head(100)

In [3]:
from sklearn.model_selection import StratifiedShuffleSplit

def get_train_test_split(data):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(data, data["target"]):
        data_train = data.loc[train_index]
        data_test = data.loc[test_index]
    return data_train, data_test

train,test = get_train_test_split(df2)

X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [4]:
X_train.head()

,season,hPos,hGSPG,hGCPG,hCSPG,hPtsPG,aPos,aGSPG,aGCPG,aCSPG,aPtsPG
3072,20,2.0,4.00,1.00,0.33,3.00,8.0,1.00,0.33,0.67,1.67
162,12,7.0,1.38,1.25,0.25,1.44,5.0,1.56,1.25,0.38,1.50
4350,23,6.0,1.76,1.24,0.29,1.59,16.0,1.18,1.88,0.18,1.00
2818,19,12.0,1.13,1.27,0.33,1.27,16.0,1.13,1.60,0.27,1.13
4549,23,3.0,1.86,0.97,0.33,1.97,16.0,0.83,1.39,0.33,1.03


In [5]:
print(X_train.shape)
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_train_norm[0]

(4256, 11)


array([0.89581175, 0.08958118, 0.17916235, 0.04479059, 0.01478089,
       0.13437176, 0.3583247 , 0.04479059, 0.01478089, 0.03000969,
       0.07480028])

In [6]:
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)

In [7]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
#model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
#model.add(tf.keras.layers.Dropout(0.5))
# model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))

In [8]:
#sgd = tf.keras.optimizers.SGD(lr=0.01,momentum=0.8,decay=0.00025)
#adadelta = tf.keras.optimizers.Adadelta()
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

class_weight = {0: 1.688,
                1: 1.747,
                2: 1}

In [9]:
p = model.fit(X_train_norm, y_train.values, epochs=15, validation_data=(X_test_norm,y_test), class_weight = class_weight)

Train on 4256 samples, validate on 1064 samples
Epoch 1/15
4256/4256 [==============================] - 1s 202us/step - loss: 1.0866 - acc: 0.4908 - val_loss: 1.0714 - val_acc: 0.4765
Epoch 2/15
4256/4256 [==============================] - 0s 71us/step - loss: 1.0635 - acc: 0.4737 - val_loss: 1.0264 - val_acc: 0.4934
Epoch 3/15
4256/4256 [==============================] - 0s 69us/step - loss: 1.0530 - acc: 0.4843 - val_loss: 1.0553 - val_acc: 0.4539
Epoch 4/15
4256/4256 [==============================] - 0s 66us/step - loss: 1.0491 - acc: 0.4774 - val_loss: 1.0672 - val_acc: 0.4445
Epoch 5/15
4256/4256 [==============================] - 0s 62us/step - loss: 1.0486 - acc: 0.4789 - val_loss: 1.0396 - val_acc: 0.4746
Epoch 6/15
4256/4256 [==============================] - 0s 70us/step - loss: 1.0476 - acc: 0.4793 - val_loss: 1.0344 - val_acc: 0.4690
Epoch 7/15
4256/4256 [==============================] - 0s 69us/step - loss: 1.0467 - acc: 0.4760 - val_loss: 1.0648 - val_acc: 0.4492
Epoch 

In [10]:
print(p)

In [11]:
train_pred = model.predict_classes(X_train_norm)
test_pred = model.predict_classes(X_test_norm)

import numpy as np

unique, counts = np.unique(train_pred, return_counts=True)
print("preds: " + str(dict(zip(unique, counts))))

unique, counts = np.unique(y_train.values, return_counts=True)
print("real: " + str(dict(zip(unique, counts))))

from sklearn.metrics import confusion_matrix as conf_matrix

print(conf_matrix(train_pred,y_train.values,labels=[0,1,2]))

preds: {0: 1809, 1: 597, 2: 1850}
real: {0: 1194, 1: 1088, 2: 1974}
[[ 743  480  586]
 [ 156  172  269]
 [ 295  436 1119]]


In [12]:
print(conf_matrix(test_pred,y_test.values))

[[175 118 156]
 [ 48  45  71]
 [ 76 109 266]]


In [13]:
df_home_wins = df.query("target == 2")
df_away_wins = df.query("target == 0")
df_draws = df.query("target == 1")
home_wins_amount = len(df_home_wins)
draw_ratio = round(home_wins_amount/len(df_draws),3)
away_win_ratio = round(home_wins_amount/len(df_away_wins),3)
home_win_ratio = 1
print("Home: {}, Away: {}, Draw: {}".format(home_win_ratio,away_win_ratio,draw_ratio))

Home: 1, Away: 1.688, Draw: 1.747
